#  Assignement 6

In [1]:

#  read California housing data
# ! wget https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
! python -m wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv



Saved under housing.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_House=pd.read_csv('housing.csv')
df_House.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# Data Prepration
keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'

In [3]:
df_House=df_House[df_House['ocean_proximity'].isin(['INLAND','<1H OCEAN'])]

In [4]:
# keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'":
df_House[df_House['ocean_proximity'].isin(['<1H OCEAN','INLAND'])]


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,<1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,<1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [5]:
df_House.head()
# Fill missing values with zeros.
df_House=df_House.fillna(0)
# Apply the log transform to median_house_value.
df_House['median_house_value']=np.log1p(df_House['median_house_value'])


# Do train/validation/test split with 60%/20%/20% distribution.
# @ SPLITTING THE DATASET:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df_House, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
y_train=df_train['median_house_value'].values
y_val=df_val['median_house_value'].values
y_test=df_test['median_house_value'].values
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']
# Use the train_test_split function and set the random_state parameter to 1.
# Use DictVectorizer(sparse=True) to turn the dataframes into matrices.
print(df_train.shape, df_val.shape, df_test.shape)



(9411, 9) (3138, 9) (3138, 9)


In [6]:
print(y_train.shape, y_val.shape, y_test.shape)

(9411,) (3138,) (3138,)


In [7]:
from sklearn.feature_extraction import DictVectorizer
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
# print(list(dv.vocabulary_.keys()))
print(dv.feature_names_)
# val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)
test_dicts = df_test.to_dict(orient='records')
# X_test = dv.transform(test_dicts)
# print(X_train.shape, X_val.shape, X_test.shape)


['households', 'housing_median_age', 'latitude', 'longitude', 'median_income', 'ocean_proximity=<1H OCEAN', 'ocean_proximity=INLAND', 'population', 'total_bedrooms', 'total_rooms']


In [12]:
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)
test_dicts = df_test.to_dict(orient='records')

Question 1<br>
Let's train a decision tree regressor to predict the median_house_value variable.<br>
<br>
Train a model with max_depth=1.<br>
Which feature is used for splitting the data?<br>
<br>
ocean_proximity<br>
total_rooms<br>
latitude<br>
population<br>

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text
dt=DecisionTreeRegressor(max_depth=1)
dt.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1)

In [9]:
# Get the feature used for splitting the data
from sklearn.tree import export_text
print(export_text(dt, feature_names=dv.feature_names_))

|--- ocean_proximity=<1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=<1H OCEAN >  0.50
|   |--- value: [12.30]



so the answer is **ocean_proximity**

Question 2<br>
Train a random forest model with these parameters:<br>
<br>

n_estimators=10<br>
random_state=1<br>
n_jobs=-1 (optional - to make training faster)<br>
What's the RMSE of this model on validation?<br>
<br>
- 0.045
- 0.245
- 0.545
- 0.845

In [10]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [13]:
# Make predictions on the validation data
y_pred = rf.predict(X_val)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE', rmse)

RMSE 0.24527740463893763


Question 3<br>
Now let's experiment with the n_estimators parameter<br>
<br>
Try different values of this parameter from 10 to 200 with step 10.<br>
Set random_state to 1.<br>
Evaluate the model on the validation dataset.<br>
After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for retrieving the answer.<br>
<br>
- 10
- 25
- 50
- 160

In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a list of n_estimators values to try
n_estimators_values = list(range(10, 201, 10))

# Initialize a list to store RMSE values for each n_estimators
rmse_values = []

# Loop over n_estimators_values
for n in n_estimators_values:
    # Train the model with the current n_estimators
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred = rf.predict(X_val)

    # Calculate the RMSE and append to rmse_values
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_values.append(rmse)

In [16]:
rmse_values


[0.24527740463893763,
 0.23904061686517195,
 0.2369950267894766,
 0.2355088433070916,
 0.23502147329312892,
 0.23458432707188806,
 0.23446619209731492,
 0.23472118653889684,
 0.23451222686838538,
 0.2343080008939288,
 0.23430472773104952,
 0.23413312513122297,
 0.23392074197851684,
 0.23367907192934007,
 0.2336055470147229,
 0.2334522392319948,
 0.23340125154040178,
 0.23357420998973866,
 0.23381388047131582,
 0.23372174449450633]

In [19]:
# Create a dictionary that maps n_estimators_values to rmse_values
n_estimators_rmse = dict(zip(n_estimators_values, rmse_values))

# Now we can print the best value of n_estimators round of till 2 decimal places

for n_estimators, rmse in n_estimators_rmse.items():
    rmse = round(rmse, 3)
    print(f'For n_estimators = {n_estimators}, RMSE = {rmse}')

For n_estimators = 10, RMSE = 0.245
For n_estimators = 20, RMSE = 0.239
For n_estimators = 30, RMSE = 0.237
For n_estimators = 40, RMSE = 0.236
For n_estimators = 50, RMSE = 0.235
For n_estimators = 60, RMSE = 0.235
For n_estimators = 70, RMSE = 0.234
For n_estimators = 80, RMSE = 0.235
For n_estimators = 90, RMSE = 0.235
For n_estimators = 100, RMSE = 0.234
For n_estimators = 110, RMSE = 0.234
For n_estimators = 120, RMSE = 0.234
For n_estimators = 130, RMSE = 0.234
For n_estimators = 140, RMSE = 0.234
For n_estimators = 150, RMSE = 0.234
For n_estimators = 160, RMSE = 0.233
For n_estimators = 170, RMSE = 0.233
For n_estimators = 180, RMSE = 0.234
For n_estimators = 190, RMSE = 0.234
For n_estimators = 200, RMSE = 0.234


The answer is **50**

Question 4<br>
Let's select the best max_depth:<br>
<br>
Try different values of max_depth: [10, 15, 20, 25]<br>
For each of these values,<br>
try different values of n_estimators from 10 till 200 (with step 10)<br>
calculate the mean RMSE<br>
Fix the random seed: random_state=1<br>
What's the best max_depth, using the mean RMSE?<br>
<br>
- 10
- 15
- 20
- 25

In [22]:
import numpy as np
best_rmse=np.inf
for max_depth in [10, 15, 20, 25]:
    for n_estimators in range(10, 210, 10):
        model = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, random_state=1)
        model.fit(X_train, y_train)
        predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        if rmse < best_rmse:
            best_rmse = rmse
            best_max_depth = max_depth
            best_n_estimators = n_estimators

print(f"The best max_depth is {best_max_depth} and the best n_estimators is {best_n_estimators} with RMSE of {best_rmse}")



The best max_depth is 10 and the best n_estimators is 40 with RMSE of 0.7534621548946745


Question 5<br>
Train the model with these parameters:<br>
n_estimators=10,<br>
max_depth=20,<br>
random_state=1,<br>
n_jobs=-1 (optional)<br>
Get the feature importance information from this model<br>
What's the most important feature (among these 4)?

In [24]:
model = RandomForestRegressor(max_depth=20, n_estimators=10,random_state=1)
model.fit(X_train, y_train)
predictions = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
#  feature importance information from the model
importances = model.feature_importances_


In [26]:
# print the feature importance information with feature names
importances_df = pd.DataFrame({'feature': dv.feature_names_, 'importance': importances})
importances_df.sort_values(by='importance', ascending=False)

,feature,importance
4,median_income,0.335611
5,ocean_proximity=<1H OCEAN,0.292473
2,latitude,0.102007
3,longitude,0.086244
6,ocean_proximity=INLAND,0.073806
1,housing_median_age,0.030317
7,population,0.027136
9,total_rooms,0.021417
8,total_bedrooms,0.015852
0,households,0.015136


Question 6 <br>
Now let's train an XGBoost model! For this question, we'll tune the eta parameter:<br>
<br>
Install XGBoost<br>
Create DMatrix for train and validation<br>
Create a watchlist<br>
Train a model with these parameters for 100 rounds:<br>
xgb_params = {<br>
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [27]:
!pip install xgboost

     ---------------------------------------- 99.7/99.7 MB 2.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
#  XGBoost
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
from sklearn.metrics import mean_squared_error
from math import sqrt

# Generate predictions
y_pred = model.predict(dvalid)

# Calculate RMSE
rmse = sqrt(mean_squared_error(y_val, y_pred))
print("RMSE: %f" % (rmse))



[0]	train-rmse:0.44350	valid-rmse:0.44250
[1]	train-rmse:0.36599	valid-rmse:0.36916
[2]	train-rmse:0.31556	valid-rmse:0.32382
[3]	train-rmse:0.28541	valid-rmse:0.29965
[4]	train-rmse:0.26573	valid-rmse:0.28417
[5]	train-rmse:0.25338	valid-rmse:0.27463


[6]	train-rmse:0.24157	valid-rmse:0.26770
[7]	train-rmse:0.23486	valid-rmse:0.26423
[8]	train-rmse:0.22668	valid-rmse:0.25820
[9]	train-rmse:0.21995	valid-rmse:0.25444
[10]	train-rmse:0.21444	valid-rmse:0.25179
[11]	train-rmse:0.21155	valid-rmse:0.25054
[12]	train-rmse:0.20729	valid-rmse:0.24791
[13]	train-rmse:0.20317	valid-rmse:0.24628
[14]	train-rmse:0.20135	valid-rmse:0.24630
[15]	train-rmse:0.19858	valid-rmse:0.24522
[16]	train-rmse:0.19580	valid-rmse:0.24445
[17]	train-rmse:0.19333	valid-rmse:0.24395
[18]	train-rmse:0.19107	valid-rmse:0.24284
[19]	train-rmse:0.18735	valid-rmse:0.24127
[20]	train-rmse:0.18524	valid-rmse:0.23978
[21]	train-rmse:0.18332	valid-rmse:0.23963
[22]	train-rmse:0.18189	valid-rmse:0.23892
[23]	train-rmse:0.18006	valid-rmse:0.23847
[24]	train-rmse:0.17869	valid-rmse:0.23847
[25]	train-rmse:0.17757	valid-rmse:0.23830
[26]	train-rmse:0.17520	valid-rmse:0.23862
[27]	train-rmse:0.17430	valid-rmse:0.23807
[28]	train-rmse:0.17281	valid-rmse:0.23793
[29]	train-rmse

In [33]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
from sklearn.metrics import mean_squared_error
from math import sqrt

# Generate predictions
y_pred = model.predict(dvalid)

# Calculate RMSE
rmse = sqrt(mean_squared_error(y_val, y_pred))
print("RMSE: %f" % (rmse))

[0]	train-rmse:0.52449	valid-rmse:0.52045
[1]	train-rmse:0.48736	valid-rmse:0.48443
[2]	train-rmse:0.45433	valid-rmse:0.45293
[3]	train-rmse:0.42533	valid-rmse:0.42550
[4]	train-rmse:0.39987	valid-rmse:0.40144
[5]	train-rmse:0.37822	valid-rmse:0.38151
[6]	train-rmse:0.35887	valid-rmse:0.36393
[7]	train-rmse:0.34177	valid-rmse:0.34838
[8]	train-rmse:0.32701	valid-rmse:0.33492
[9]	train-rmse:0.31411	valid-rmse:0.32333
[10]	train-rmse:0.30326	valid-rmse:0.31427
[11]	train-rmse:0.29355	valid-rmse:0.30615
[12]	train-rmse:0.28519	valid-rmse:0.29922
[13]	train-rmse:0.27760	valid-rmse:0.29269
[14]	train-rmse:0.27116	valid-rmse:0.28796
[15]	train-rmse:0.26538	valid-rmse:0.28380
[16]	train-rmse:0.26012	valid-rmse:0.27970
[17]	train-rmse:0.25583	valid-rmse:0.27661
[18]	train-rmse:0.25203	valid-rmse:0.27388
[19]	train-rmse:0.24818	valid-rmse:0.27123
[20]	train-rmse:0.24512	valid-rmse:0.26882
[21]	train-rmse:0.24149	valid-rmse:0.26669
[22]	train-rmse:0.23895	valid-rmse:0.26478
[23]	train-rmse:0.235